# 开始学习Prompt Engineering
by HelloGPX

本手册包含学习提示工程的示例和练习。我们将通过本代码示例学习百度文心一言和Azure OpenAI的基础Prompt Engineering

## 1. Prompt Engineering的准备工作

- 导入python包：openai, qianfan
- 查看自己的百度千帆大模型账户上的**api key**和**secrect key**，或者Azure OpenAI上的**api key**

参考资料
- 百度千帆大模型: https://qianfan.readthedocs.io/en/stable/README.html
- Azure OpenAI: https://learn.microsoft.com/zh-CN/azure/ai-services/openai/

In [1]:
!pip install openai
!pip install qianfan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 6.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 kB 6.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.2/329.2 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.0
    Uninstalling python-dotenv-1.0.0:
      Successfully uninstalled python-dotenv-1.0.0


In [2]:
import qianfan
from openai import AzureOpenAI
import IPython

# 全局配置
temperature = 0.1
max_tokens = 2000

# Qianfan 参数配置
QIANFAN_API_KEY = "" # @千帆大模型的api key
QIANFAN_SECRET_KEY = "" #@千帆大模型的secret key

# Azure OpenAI 参数配置
AOAI_API_KEY = "" #@azure openai的api key
API_VERSION = "2023-12-01-preview"
AZURE_ENDPOINT = "" #@azure openai的endpoint, 格式通常为 https://your-service-name.openai.azure.com/

qianfan_chatcompletion = qianfan.ChatCompletion(ak=QIANFAN_API_KEY,sk=QIANFAN_SECRET_KEY)
qianfan_completion = qianfan.Completion(ak=QIANFAN_API_KEY,sk=QIANFAN_SECRET_KEY)
aoai_client = AzureOpenAI(
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AOAI_API_KEY,
)

## 定义简单函数
- ChatCompletion和Completion的函数
- OpenAI的chat模型可以接受[system, user, assistant]的角色定义，文心一言的chat模型只能接受[user, assistant]，因此需要一个转换函数

In [3]:
import copy
def convert_system_message(messages:dict) -> dict:
    new_messages = copy.deepcopy(messages)
    new_messages[0]["role"] = "user"
    new_messages[0]["content"] = "系统设置：" + new_messages[0]["content"]
    new_ai_message = {"role":"assistant", "content":"当然可以。请告诉我您的输入。"}
    new_messages.insert(1, new_ai_message)
    return new_messages

def get_chat_completion(messages:list, model_type:str) -> str:
    if model_type == "openai":
        response = aoai_client.chat.completions.create(
            model="gpt-35-turbo",
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    
    elif model_type == "baidu":
        response=qianfan_chatcompletion.do(
            messages=convert_system_message(messages),
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response['body']['result']
    
def get_completion(prompt:str, model_type:str) -> str:
    if model_type == "openai":
        response = aoai_client.completions.create(
            model="text-davinci-003",
            prompt=prompt,
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response.choices[0].text
    elif model_type == "baidu":
        response = qianfan_completion.do(
            prompt=prompt, 
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response['body']['result']

## 2. ChatGPT和百度文心一言的基础调用方式
### 2.1 Completion

In [4]:
prompt = """请将如下英文翻译为中文，要求保留其中的英文转有名词原词为英文。
输出格式为：{"原文":"英文原文内容", "翻译":"翻译后的中文内容"}

原文：hello, how can I help you today?
{"原文":"hello, how can I help you today?", "翻译":"今天有什么我可以帮助您的吗？"}

原文：What the wether like today in Shanghai?
"""

In [5]:
response = get_completion(prompt=prompt, model_type="openai")
IPython.display.Markdown(response)

{"原文":"What the wether like today in Shanghai?", "翻译":"今天上海的天气怎么样？"}

In [6]:
response = get_completion(prompt=prompt, model_type="baidu")
IPython.display.Markdown(response)

[INFO] [02-15 09:38:24] logging.py:68 [t:134131100473152]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-15 09:38:24] logging.py:68 [t:134131100473152]: trying to refresh access_token for ak `rGbMT1***`
[INFO] [02-15 09:38:26] logging.py:68 [t:134131100473152]: sucessfully refresh access_token


{"原文":"What the wether like today in Shanghai?", "翻译":"上海今天天气怎么样？"}

## 2.2 Chat Completion

In [7]:
messages = [
    {"role":"system", "content":'请将如下英文翻译为中文，要求保留其中的英文转有名词原词为英文。输出格式为：{"原文":"英文原文内容", "翻译":"翻译后的中文内容"}'},
    {"role":"user", "content":"hello, how can I help you today?"},
    {"role":"assistant", "content":'{"原文":"hello, how can I help you today?", "翻译":"今天有什么我可以帮助您的吗？"}'},
    {"role":"user", "content":"What the wether like today in Shanghai?"}
]

In [8]:
response = get_chat_completion(messages=messages, model_type="openai")
IPython.display.Markdown(response)

{"原文":"What the wether like today in Shanghai?", "翻译":"上海今天的天气如何？"}

In [9]:
response = get_chat_completion(messages=messages, model_type="baidu")
IPython.display.Markdown(response)

[INFO] [02-15 09:38:29] logging.py:68 [t:134131100473152]: requesting llm api endpoint: /chat/eb-instant


{"原文":"What the weather like today in Shanghai?", "翻译":"今天上海的天气怎么样？"}

# 3. 基础Prompt功能案例
- Zero-shot prompts
- Few-shot prompts

## 3.1 代码生成 （Zero-shot prompts）

In [10]:
prompt = """使用python及其pandas库完成如下任务：当前有一个名为table的Dataframe，包含有列["student name", "grade", "score", "age"]。
我需要你输出一段python代码，将其中age在13~16范围的学生的成绩求平均数。
请直接输出代码："""

In [11]:
response = get_completion(prompt=prompt, model_type="openai")
IPython.display.Markdown(response)



table[(table['age'] >= 13) & (table['age'] <= 16)]['score'].mean()

In [12]:
response = get_completion(prompt=prompt, model_type="baidu")
IPython.display.Markdown(response)

[INFO] [02-15 09:38:31] logging.py:68 [t:134131100473152]: requesting llm api endpoint: /chat/eb-instant


可以使用pandas库中的筛选和求平均函数来完成这个任务。以下是一段可能的代码：


```python
import pandas as pd

# 假设你的DataFrame名为table
table = pd.DataFrame({
    'student name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'grade': [1, 2, 3, 4, 5],
    'score': [85, 90, 78, 92, 88],
    'age': [14, 15, 13, 16, 14]
})

# 使用条件筛选出年龄在13到16之间的学生
age_range = table[(table['age'] >= 13) & (table['age'] <= 16)]

# 计算这些学生的平均成绩
average_score = age_range['score'].mean()

print(f"平均成绩为: {average_score}")
```
这段代码首先创建了一个包含所有数据的DataFrame，然后使用条件筛选找出年龄在13到16之间的学生，再对这些学生的成绩求平均值。最后，打印出平均成绩。请注意，这段代码假设年龄列的数据类型为整数。如果数据类型不是整数，你可能需要在使用条件筛选之前先进行数据类型转换。

## 3.2 简单逻辑判断 （Zero-shot Prompts）

In [13]:
prompt = """请判断所给的一组数字之和是奇数还是偶数。
数字集合：[2,23,4]
答案：
"""

In [14]:
response = get_completion(prompt=prompt, model_type="openai")
IPython.display.Markdown(response)

偶数。

In [15]:
response = get_completion(prompt=prompt, model_type="baidu")
IPython.display.Markdown(response)

[INFO] [02-15 09:38:35] logging.py:68 [t:134131100473152]: requesting llm api endpoint: /chat/eb-instant


这组数字之和是偶数。

因为这组数字中包含了两个2和一个4，它们的和是2 + 23 + 4 = 29。偶数是可以被2整除的数字，而29可以被2整除，所以这组数字之和是偶数。

## 3.2 简单逻辑判断 （Few-shot Prompts）

In [16]:
prompt = """请判断所给的一组数字之和是奇数还是偶数。
数字集合：[2,23,4]
答案：奇数

数字集合：[22, 22, 2]
答案：
"""

In [17]:
response = get_completion(prompt=prompt, model_type="openai")
IPython.display.Markdown(response)

偶数

In [18]:
response = get_completion(prompt=prompt, model_type="baidu")
IPython.display.Markdown(response)

[INFO] [02-15 09:38:37] logging.py:68 [t:134131100473152]: requesting llm api endpoint: /chat/eb-instant


偶数